# 智能体（Agent）：构建一个与外部工具交互的智能体。

LangChain 支持创建**智能体（agents）**，即使用大语言模型（LLMs）作为推理引擎来决定应采取哪些操作以及执行这些操作所需的输入的系统。  
在执行完操作后，结果可以反馈给 LLM，以判断是否需要执行更多操作，或者是否可以结束任务。这一过程通常通过 **工具调用（tool-calling）** 来实现。

在本教程中，我们将构建一个可以与**搜索引擎**交互的智能体。你可以向这个智能体提问，观察它调用搜索工具，并与它进行多轮对话。

---

### 端到端智能体示例

下面的代码片段展示了一个**功能完整的智能体**，它使用大语言模型来决定应该调用哪些工具。  
该智能体配备了一个通用的搜索工具，并具有**会话记忆能力**，这意味着它可以作为一个支持多轮对话的聊天机器人使用。

如果你只是想快速获取一段可运行的代码并开始使用，可以直接使用以下模板。

In [1]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    _ = load_dotenv()
except ImportError:
    pass

if not os.environ.get("DASHSCOPE_API_KEY"):
  os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [2]:
from langchain_community.chat_models.tongyi import ChatTongyi

model = ChatTongyi(
    streaming=True,
    name="qwen-turbo"
)

In [3]:
# %pip install --upgrade --quiet google-search-results

In [4]:
from langchain_community.utilities import SerpAPIWrapper
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

# Create the agent
memory = MemorySaver()
# params = {
#     "engine": "baidu",
# }

params = {
    "engine": "bing",
    "gl": "cn",        # 地理位置改为中国
    "hl": "zh-cn",     # 语言改为简体中文
}
search = SerpAPIWrapper(params=params)

@tool
def search_tool(query: str) -> str:
    """Search the web using SerpAPI"""
    return search.run(query)
    
tools = [search_tool]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [5]:
from langchain_core.messages import SystemMessage

# Use the agent
config = {"configurable": {"thread_id": "abc125"}}

messages = [
    SystemMessage(content="如果模型调用工具tools，工具如果返回的是url网页连接，必须访问能打开的url链接，并将网页的内容摘要输出来。"),
]

input_message = {
    "role": "user",
    "content": "嗨，我是张三，住在天津。",
}

inp = messages + [input_message]
for step in agent_executor.stream(
    {"messages": inp}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

嗨，我是张三，住在天津。
================================== Ai Message ==================================

你好张三，很高兴认识你！你是哪里人呢？


In [6]:
input_message = {
    "role": "user",
    "content": "我住的地方天气怎么样?",
}
inp = messages + [input_message]
for step in agent_executor.stream(
    {"messages": inp}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

我住的地方天气怎么样?
================================== Ai Message ==================================
Tool Calls:
  search_tool (call_f5845d767f894fcb9c5847call_f5845d767f894fcb9c5847call_f5845d767f894fcb9c5847call_f5845d767f894fcb9c5847)
 Call ID: call_f5845d767f894fcb9c5847call_f5845d767f894fcb9c5847call_f5845d767f894fcb9c5847call_f5845d767f894fcb9c5847
  Args:
    query: weather in Tianjin
================================= Tool Message =================================
Name: search_tool

['Be prepared with the most accurate 10-day forecast for Wilmington, DE with highs, lows, chance of precipitation from The Weather Channel and Weather.com', 'Wilmington, DE Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.', 'A chance of showers. Cloudy, with a low around 62. Northeast wind around 5 mph. Chance of precipitation is 50%. New precipitation amounts of less than a 

In [8]:
# %pip install --upgrade --quiet langgraph langchain-tavily langgraph-checkpoint-sqlite

In [9]:
query = "你好!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'你好! 😊 有什么我可以帮助你的?'

我们现在可以看看如何让这个模型具备调用工具的能力。为了实现这一点，我们使用 .bind_tools 方法，让语言模型了解这些工具的存在和使用方式。

In [10]:
model_with_tools = model.bind_tools(tools)

现在我们可以调用这个模型了。我们首先用一条普通消息来调用它，看看它是如何回应的。我们可以同时查看 content 字段和 tool_calls 字段的内容。

In [11]:
query = "你好!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 你好! 有什么我可以帮助您的?

Tool calls: []


现在，让我们尝试使用一些需要调用工具的输入来调用模型。

In [12]:
query = "我请查询天津的天气?"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'search_tool', 'args': {'query': '天津天气'}, 'id': 'call_bad566de9c214096bb02cfcall_bad566de9c214096bb02cfcall_bad566de9c214096bb02cfcall_bad566de9c214096bb02cf', 'type': 'tool_call'}]


我们可以看到，现在没有文本内容，但出现了一个工具调用！它希望我们调用 **Serp Search** 工具。

但这一步**还没有真正调用**该工具——它只是告诉我们需要调用哪个工具。为了实际执行这个工具调用，我们需要构建我们的**智能体（agent）**。

### 创建智能体（Agent）

现在我们已经定义好了工具（tools）和语言模型（LLM），接下来就可以创建智能体了。

我们将使用 **LangGraph** 来构建这个智能体。目前，我们会使用一个**高级接口**来创建智能体。但 LangGraph 的优势在于：这个高级接口的背后是一个**低级、高度可控的 API**，如果你需要自定义智能体的逻辑，可以基于它进行深入调整。

现在，我们可以使用 LLM 和工具来初始化智能体。

> ⚠️ 注意：我们传入的是 `model`，而不是 `model_with_tools`。这是因为 `create_react_agent` 会在底层自动帮我们调用 `.bind_tools` 方法。你不需要手动绑定工具。

In [13]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

### 运行智能体（Agent）

现在我们可以使用几个查询来运行这个智能体了！请注意，目前这些查询都是**无状态的**（也就是说，智能体不会记住之前的交互）。

智能体将在交互结束后返回最终的状态（其中包含了所有的输入信息，稍后我们会看到如何只获取输出结果）。

首先，我们来看一个**不需要调用工具**的示例，看看智能体会如何回应：

In [14]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I assist you today?


现在，我们来尝试运行一个**需要调用工具**的示例，看看智能体的实际表现。

In [15]:
input_message = {"role": "user", "content": "搜索天津的天气"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

搜索天津的天气
================================== Ai Message ==================================
Tool Calls:
  search_tool (call_e7ba2342dcdb4b20b2cba7call_e7ba2342dcdb4b20b2cba7call_e7ba2342dcdb4b20b2cba7call_e7ba2342dcdb4b20b2cba7)
 Call ID: call_e7ba2342dcdb4b20b2cba7call_e7ba2342dcdb4b20b2cba7call_e7ba2342dcdb4b20b2cba7call_e7ba2342dcdb4b20b2cba7
  Args:
    query: 天津天气
================================= Tool Message =================================
Name: search_tool

['天津天气预报，及时准确发布中央气象台天气信息，便捷查询天津今日天气，天津周末天气，天津一周天气预报，天津蓝天预报，天津天气预报，天津40日天气预报，还提供天津的生活指数、 …', '天津市; 天津天气预报 ... 每日天气提示 . 天气公报 . 环境气象公报 . 全球灾害性天气监测月报 . fy-4b . 雷达拼图 . 推荐服务 . 世界气象中心(北京) 台风网 . 气象期刊 . 航空气象 . 沙尘网 . 科技合作 .', '每小时本地天气预报、天气情况、降水、露点、湿度、大风 - 尽在 Weather.com 和 The Weather Channel', '今日天气：天津市，多云,21℃~28℃,西北风2级，当前温度23℃。', '围观天气提供天津天气预报、未来天津7天、15天气天气，方便大家查询天津天气预报包括温度、降雨以及空气质量pm2.5的实时数据，帮助您及时根据天气情况安排工作以及生活，查天气就 …', '天津天津天气预报，及时准确发布中央气象台天气信息，便捷查询北京今

### 流式传输消息（Streaming Messages）

我们已经了解了如何通过 `.invoke` 方法调用智能体来获取最终的响应。但如果智能体在执行过程中需要完成多个步骤，这可能需要一定的时间。

为了展示中间的处理进度，我们可以**在消息生成时就将它们流式返回**给用户。这样可以实现实时反馈，提升用户体验。

In [16]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

搜索天津的天气
================================== Ai Message ==================================
Tool Calls:
  search_tool (call_8cb9164defac4f00901782call_8cb9164defac4f00901782call_8cb9164defac4f00901782call_8cb9164defac4f00901782)
 Call ID: call_8cb9164defac4f00901782call_8cb9164defac4f00901782call_8cb9164defac4f00901782call_8cb9164defac4f00901782
  Args:
    query: 天津天气
================================= Tool Message =================================
Name: search_tool

['天津天气预报，及时准确发布中央气象台天气信息，便捷查询天津今日天气，天津周末天气，天津一周天气预报，天津蓝天预报，天津天气预报，天津40日天气预报，还提供天津的生活指数、 …', '天津市; 天津天气预报 ... 每日天气提示 . 天气公报 . 环境气象公报 . 全球灾害性天气监测月报 . fy-4b . 雷达拼图 . 推荐服务 . 世界气象中心(北京) 台风网 . 气象期刊 . 航空气象 . 沙尘网 . 科技合作 .', '每小时本地天气预报、天气情况、降水、露点、湿度、大风 - 尽在 Weather.com 和 The Weather Channel', '今日天气：天津市，多云,21℃~28℃,西北风2级，当前温度23℃。', '围观天气提供天津天气预报、未来天津7天、15天气天气，方便大家查询天津天气预报包括温度、降雨以及空气质量pm2.5的实时数据，帮助您及时根据天气情况安排工作以及生活，查天气就 …', '天津天津天气预报，及时准确发布中央气象台天气信息，便捷查询北京今

### 流式传输 Token（Streaming Tokens）

除了可以流式返回整条消息之外，**逐个 Token 地流式返回**也是非常有用的。我们可以通过设置 `stream_mode="messages"` 来实现这一功能。

> ⚠️ 注意  以下代码中使用了 `message.text()` 方法，该方法要求 `langchain-core` 版本至少为 **0.3.37** 或更高。  

In [17]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

根|据最新搜索结果|，天津今天的天气|是多云，|气温范围在2|1℃到2|8℃之间，|西北风2级|。当前温度为|23℃。|

### 添加记忆功能（Adding in Memory）

如前所述，当前这个智能体是**无状态的（stateless）**，也就是说它不会记住之前的交互内容。

为了给它添加记忆功能，我们需要传入一个 **checkpointer（检查点存储器）**。  
当我们传入了 checkpointer 后，在调用智能体时还需要同时传入一个 `thread_id`（线程ID），这样智能体就知道它是从**哪一个对话线程**中恢复执行的。

In [18]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [19]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [20]:
input_message = {"role": "user", "content": "你好 我是张三 住在天津。"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

你好 我是张三 住在天津。
================================== Ai Message ==================================

你好张三！请问有什么我可以帮助您的？


In [21]:
input_message = {"role": "user", "content": "我叫什么 还记得吗?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

我叫什么 还记得吗?
================================== Ai Message ==================================

抱歉，作为一个AI助手，我没有记忆功能，所以无法记住之前的对话内容或个人信息。不过，您刚才提到您的名字是张三，对吧？如果您有任何问题或需要帮助，请随时告诉我！


In [23]:
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "我叫什么 还记得吗?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

我叫什么 还记得吗?
================================== Ai Message ==================================

抱歉，作为一个AI助手，我没有记忆功能，所以我无法记住之前的对话内容或个人信息。你可以告诉我你的名字，我会尽力帮助你！
